In [229]:
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw, ImageOps
import string
import numpy as np
import cv2
from tqdm import tqdm

In [2]:
#set the seed
np.random.seed(seed=123)

# Function to generate watermarks on images 

In [217]:
#list of printable characters that could appear in a watermark
prints = list(string.printable)[0:84]

def gen_watermark(im_str, train = 1):
    
    #open all of the images from the VOC2008 dataset as jpegs
    img = Image.open("raw_data/{}".format(im_str))
    
    #save original image as np array to 'RAWS' directory
    img.save('data/no-watermark/{}'.format(im_str))
    
    #generate random font size between 30-100
    font_size = np.random.randint(low = 20, high = 100)
    
    #create the watermark font for the image
    font = ImageFont.truetype("abel-regular.ttf", font_size) 
    
    #generate image to hold the watermark text object
    img_temp = Image.new('L', (350,350))
    
    #create the watermark text, of random length, using random printable characters
    text_str = np.random.choice(prints, np.random.randint(low=5, high = 10))
    text_str = "".join(text_str)
    
    #draw on temporary image with text
    draw_temp = ImageDraw.Draw(img_temp) 
    
    #generate a random integer for the opacity argument (fill)
    opac = np.random.randint(low=50, high=200)
    
    #insert text onto the temporary image
    draw_temp.text((0, 0), text_str,  font=font, fill=opac)
    
    #generate a random integer for rotation:
    rot_int = np.random.randint(low = 0, high = 20)
    
    #rotate the text on the temporary image
    rotated_text = img_temp.rotate(rot_int,  expand=1)
    
    #default color of watermark set to white; change if desired
    col_1 = (255,255,255)
    col_2 = (255,255,255)
    
    #generate a random location for the watermark on the image
    rand_loc = [np.random.randint(low=0,high=int(img.size[0]/2)), np.random.randint(low=0,high=int(img.size[1]/2))]
    
    #merge the temporary image with text with the image passed in 
    #third tuple also needs to be random: controls the location of the img
    img.paste(ImageOps.colorize(rotated_text, col_1, col_2), rand_loc,  rotated_text)
    
    #this yeilds a new image with a watermark
    #save this jpeg with a watermark to the WATS directory
    img.save('data/watermark/{}'.format(im_str))
#     return img

In [222]:
# nwpath = "data/no-watermark"
# wpath = "data/watermark"
# os.mkdir(nwpath)
# os.mkdir(wpath)

In [223]:
#generate 
im_list = os.listdir("raw_data")

#run time is about 5-6 minutes for all 5000 images:
for ind, img in enumerate(im_list):
#     add the watermark to the image and save to 'watered' directory
    wat_img = gen_watermark(img)

In [224]:
im_list = os.listdir("data\watermark")
len(im_list)

5096

In [225]:
data_dir = []
data_dir = 'data'
image_exts = ['jpg', 'jpeg', 'png']

In [230]:
from struct import unpack

marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]            
            if len(data)==0:
                break        


bads = []
for image_class in os.listdir(data_dir):
    for image in tqdm(os.listdir(os.path.join(data_dir, image_class))):
        image_path = os.path.join(data_dir, image_class, image)
        image = JPEG(image_path) 
        try:
          image.decode()   
        except:
          bads.append(image_path)


    for name in bads:
#         os.remove(name)
        print(name)

100%|█████████████████████████████████████████████████████████████████████████████| 5096/5096 [00:26<00:00, 194.74it/s]


In [240]:
def remove_corrupted_images(directory, bad):
    for image_class in os.listdir(directory):
        for image in tqdm(os.listdir(os.path.join(directory, image_class))):
            image_path = os.path.join(directory, image_class, image)
            try:
                img = cv2.imread(image_path)
#                 tip = imghdr.what(image_path)
#                 if tip not in image_exts:
#                     print(f"image extension not valid for path {image_path}")
#                     bad.append(image_path)
# #                     os.remove(image_path)
            except:
                print(f"Corrupted image at {image_path}")
                bad.append(image_path)
#                 os.remove(image_path)

In [ ]:
corrupted = []
remove_corrupted_images(data_dir, corrupted)